In [1]:
import pandas as pd
import numpy as np
import pickle as p
from helper_functions import optimize_memory

In [2]:
# config
master_data_input_path = 'data_all.csv'
item_cost_input_path = 'FullItemRec.csv'
model_path = 'models/'

# load data
sales_data = pd.read_csv(master_data_input_path, skiprows=1, usecols=['Wk','Tier','Store','Item_ID','Qty_','Price_'])
sales_data = optimize_memory(sales_data)
sales_data.dropna(axis=0, inplace=True)
sales_data.isnull().sum().sum()

0

## `/train`

In [48]:
import requests
import json
from threading import Thread
import time

# url = 'https://fillet.azurewebsites.net/train/'
url = 'http://localhost:5000/train/'
sales_data.to_parquet('data_all.parquet')
files = {'data': open('data_all.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 'shan', 'modeltype': 'default'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

time.sleep(10)
files['data'].close()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/urllib3/connectionpool.py", line 677, in urlopen
    chunked=chunked,
  File "/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/anaconda3/envs/capstone/lib/python3.7/http/client.py", line 1344, in getresponse
    response.begin()
  File "/opt/anaconda3/envs/capstone/lib/python3.7/http/client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "/opt/anaconda3/envs/capstone/lib/python3.7/http/client.py", line 275, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Rem

## `/Optimize`

In [20]:
import json
with open('constraints_2.json') as f:
    constraints = json.load(f)

In [44]:
import requests
import json

url = 'http://localhost:5000/optimize/'
    
payload = {'project_id': 'shan',
           'constraints': constraints, # two parts, first part constraint, second part price range limits
           'population': 300,
           'max_epoch': 100
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [45]:
best_prices = r.json()
best_prices

{'price_cols': ['Price_1',
  'Price_2',
  'Price_11',
  'Price_12',
  'Price_21',
  'Price_22',
  'Price_31',
  'Price_101',
  'Price_102',
  'Price_196',
  'Price_197',
  'Price_200',
  'Price_201',
  'Price_202',
  'Price_203',
  'Price_204',
  'Price_281',
  'Price_284',
  'Price_350',
  'Price_1002',
  'Price_1003',
  'Price_1004',
  'Price_1152',
  'Price_1153',
  'Price_1195',
  'Price_1203',
  'Price_1204',
  'Price_1205',
  'Price_1221',
  'Price_1225',
  'Price_1230',
  'Price_1231',
  'Price_1232',
  'Price_1502',
  'Price_1503',
  'Price_1504',
  'Price_1615',
  'Price_2256',
  'Price_2257',
  'Price_2258',
  'Price_2259',
  'Price_2313',
  'Price_2315',
  'Price_2316',
  'Price_2501',
  'Price_2511',
  'Price_2521',
  'Price_2531',
  'Price_2541',
  'Price_2551',
  'Price_2571',
  'Price_2581',
  'Price_2588',
  'Price_2589',
  'Price_2701',
  'Price_2721',
  'Price_2731',
  'Price_2736',
  'Price_2867',
  'Price_2887',
  'Price_3045',
  'Price_3046',
  'Price_3047',
  'Pri

## `/detect_conflict`

In [69]:
import requests
import json

url = 'http://localhost:5000/detect_conflict/'
with open('constraints_2.json') as f:
    constraints = json.load(f)
    
payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [70]:
r.json()['conflict']

'No conflict'